![trainers in a store](trainers.jpg)

Sports clothing and athleisure attire is a huge industry, worth approximately [$193 billion in 2021](https://www.statista.com/statistics/254489/total-revenue-of-the-global-sports-apparel-market/) with a strong growth forecast over the next decade! 

In this notebook, you will undertake the role of a product analyst for an online sports clothing company. The company is specifically interested in how it can improve revenue. You will dive into product data such as pricing, reviews, descriptions, and ratings, as well as revenue and website traffic, to produce recommendations for its marketing and sales teams.  

You've been provided with four datasets to investigate:

#  brands.csv

| Columns | Description |
|---------|-------------|
| `product_id` | Unique product identifier |
| `brand` | Brand of the product | 

# finance.csv

| Columns | Description |
|---------|-------------|
| `product_id` | Unique product identifier |
| `listing_price` | Original price of the product | 
| `sale_price` | Discounted price of the product |
| `discount` | Discount off the listing price, as a decimal | 
| `revenue` | Revenue generated by the product |

# info.csv

| Columns | Description |
|---------|-------------|
| `product_name` | Name of the product | 
| `product_id` | Unique product identifier |
| `description` | Description of the product |

# reviews.csv

| Columns | Description |
|---------|-------------|
| `product_id` | Unique product identifier |
| `rating` | Average product rating | 
| `reviews` | Number of reviews for the product |

In order to complete the project you will need to merge the datasets and drop null values, along with answering the questions below.

What is the volume of products and average revenue for Adidas and Nike products based on listing price quartiles?

- Label products priced up to quartile one as "Budget", quartile two as "Average", quartile three as "Expensive", and quartile four as "Elite".
- Store as a pandas DataFrame called adidas_vs_nike containing the following columns: "brand", "price_label", "num_products", and "mean_revenue". All numeric values should be rounded to two decimal places.

Do any differences exist between the word count of a product's description and its mean rating?

- Split product description length into bins of 100 characters and calculate the average rating and number of reviews.
- Store the results as a pandas DataFrame called description_lengths containing the following columns: "description_length", "mean_rating", "num_reviews", again rounding numeric values to two decimal places.

In [28]:
import pandas as pd

brands = pd.read_csv("brands.csv") 
finance = pd.read_csv("finance.csv")
info = pd.read_csv("info.csv")
reviews = pd.read_csv("reviews.csv")

### Informacje o danych

In [29]:
brands.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3179 entries, 0 to 3178
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   product_id  3179 non-null   object
 1   brand       3120 non-null   object
dtypes: object(2)
memory usage: 49.8+ KB


In [30]:
finance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3179 entries, 0 to 3178
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   product_id     3179 non-null   object 
 1   listing_price  3120 non-null   float64
 2   sale_price     3120 non-null   float64
 3   discount       3120 non-null   float64
 4   revenue        3120 non-null   float64
dtypes: float64(4), object(1)
memory usage: 124.3+ KB


In [31]:
info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3179 entries, 0 to 3178
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   product_name  3120 non-null   object
 1   product_id    3179 non-null   object
 2   description   3117 non-null   object
dtypes: object(3)
memory usage: 74.6+ KB


In [32]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3179 entries, 0 to 3178
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   product_id  3179 non-null   object 
 1   rating      3120 non-null   float64
 2   reviews     3120 non-null   float64
dtypes: float64(2), object(1)
memory usage: 74.6+ KB


### Mergowanie

In [33]:
merge_df = info.merge(finance, on='product_id')
merge_df = merge_df.merge(reviews, on='product_id')
merge_df = merge_df.merge(brands, on='product_id')
merge_df.dropna(inplace=True)

In [34]:
merge_df.head()

,product_name,product_id,description,listing_price,sale_price,discount,revenue,rating,reviews,brand
1,Women's adidas Originals Sleek Shoes,G27341,"A modern take on adidas sport heritage, tailor...",75.99,37.99,0.5,1641.17,3.3,24.0,Adidas
2,Women's adidas Swim Puka Slippers,CM0081,These adidas Puka slippers for women's come wi...,9.99,5.99,0.4,398.93,2.6,37.0,Adidas
3,Women's adidas Sport Inspired Questar Ride Shoes,B44832,"Inspired by modern tech runners, these women's...",69.99,34.99,0.5,2204.37,4.1,35.0,Adidas
4,Women's adidas Originals Taekwondo Shoes,D98205,This design is inspired by vintage Taekwondo s...,79.99,39.99,0.5,5182.70,3.5,72.0,Adidas
5,Women's adidas Sport Inspired Duramo Lite 2.0 ...,B75586,Refine your interval training in these women's...,47.99,19.20,0.6,1555.20,1.0,45.0,Adidas


In [35]:
merge_df['brand'].value_counts()

brand
Adidas    2575
Nike       542
Name: count, dtype: int64

In [36]:
merge_df['product_id'].nunique()

3117

### Pytanie 1

What is the volume of products and average revenue for Adidas and Nike products based on listing price quartiles?

- Label products priced up to quartile one as "Budget", quartile two as "Average", quartile three as "Expensive", and quartile four as "Elite".
- Store as a pandas DataFrame called adidas_vs_nike containing the following columns: "brand", "price_label", "num_products", and "mean_revenue". All numeric values should be rounded to two decimal places.

In [37]:
merge_df['price_label'] = pd.qcut(merge_df['listing_price'], q=4, labels=["Budget", "Average", "Expensive", "Elite"])
#num_product, mean_revenue
adidas_vs_nike = merge_df.groupby(['brand', 'price_label']).agg(
    num_product = ('product_id', 'count'),
    mean_value = ('revenue', 'mean')
).round(2)

print(adidas_vs_nike)

                    num_product  mean_value
brand  price_label                         
Adidas Budget               574     2015.68
       Average              655     3035.30
       Expensive            759     4621.56
       Elite                587     8302.78
Nike   Budget               357     1596.33
       Average                8      675.59
       Expensive             47      500.56
       Elite                130     1367.45


/tmp/ipykernel_23497/4088249900.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  adidas_vs_nike = merge_df.groupby(['brand', 'price_label']).agg(


### Pytanie 2

Do any differences exist between the word count of a product's description and its mean rating?

- Split product description length into bins of 100 characters and calculate the average rating and number of reviews.
- Store the results as a pandas DataFrame called description_lengths containing the following columns: "description_length", "mean_rating", "num_reviews", again rounding numeric values to two decimal places.

In [38]:
merge_df['description_length'] = merge_df['description'].str.len()
merge_df

,product_name,product_id,description,listing_price,sale_price,discount,revenue,rating,reviews,brand,price_label,description_length
1,Women's adidas Originals Sleek Shoes,G27341,"A modern take on adidas sport heritage, tailor...",75.99,37.99,0.5,1641.17,3.3,24.0,Adidas,Expensive,175
2,Women's adidas Swim Puka Slippers,CM0081,These adidas Puka slippers for women's come wi...,9.99,5.99,0.4,398.93,2.6,37.0,Adidas,Budget,172
3,Women's adidas Sport Inspired Questar Ride Shoes,B44832,"Inspired by modern tech runners, these women's...",69.99,34.99,0.5,2204.37,4.1,35.0,Adidas,Expensive,264
4,Women's adidas Originals Taekwondo Shoes,D98205,This design is inspired by vintage Taekwondo s...,79.99,39.99,0.5,5182.70,3.5,72.0,Adidas,Expensive,288
5,Women's adidas Sport Inspired Duramo Lite 2.0 ...,B75586,Refine your interval training in these women's...,47.99,19.20,0.6,1555.20,1.0,45.0,Adidas,Average,221
...,...,...,...,...,...,...,...,...,...,...,...,...
3174,Nike Tiempo Legend 8 Academy TF,AT6100-606,The Nike Tiempo Legend 8 Academy TF takes the ...,0.00,64.95,0.0,0.00,0.0,0.0,Nike,Budget,146
3175,Nike React Metcon AMP,CT9155-063,The Nike React Metcon AMP takes the stability ...,0.00,139.95,0.0,251.91,3.0,1.0,Nike,Budget,378
3176,Air Jordan 8 Retro,CI1236-100,The Air Jordan 8 Retro recaptures the memorabl...,159.95,127.97,0.0,230.35,5.0,1.0,Nike,Elite,204
3177,Nike Air Max 98,AH6799-300,The Nike Air Max 98 features the OG design lin...,0.00,169.95,0.0,1223.64,4.0,4.0,Nike,Budget,240


In [39]:
bins = [0, 100, 200, 300, 400, 500, 600, 700]
labels = ["100", "200", "300", "400", "500", "600", "700"]

merge_df['description_length'] = pd.cut(merge_df['description_length'], bins=bins, labels=labels)

merge_df_grouped = merge_df.groupby('description_length', as_index=False).agg(
    mean_rating = ('rating', 'mean'),
    num_reviews = ('reviews', 'count')
).round(2)

print(merge_df_grouped)

  description_length  mean_rating  num_reviews
0                100         2.26            7
1                200         3.19          526
2                300         3.28         1785
3                400         3.29          651
4                500         3.35          118
5                600         3.12           15
6                700         3.65           15


/tmp/ipykernel_23497/4265708781.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  merge_df_grouped = merge_df.groupby('description_length', as_index=False).agg(


# Solution

In [40]:
# Start coding here... 
import pandas as pd

# Read in the data
info = pd.read_csv("info.csv")
finance = pd.read_csv("finance.csv")
reviews = pd.read_csv("reviews.csv")
brands = pd.read_csv("brands.csv")

# Merge the data and drop null values
merged_df = info.merge(finance, on="product_id")
merged_df = merged_df.merge(reviews, on="product_id")
merged_df = merged_df.merge(brands, on="product_id")
merged_df.dropna(inplace=True)

# Add price labels based on listing_price quartiles
merged_df["price_label"] = pd.qcut(merged_df["listing_price"], q=4, labels=["Budget", "Average", "Expensive", "Elite"])

# Group by brand and price_label to get volume and mean revenue
adidas_vs_nike = merged_df.groupby(["brand", "price_label"], as_index=False).agg(
    num_products=("price_label", "count"), 
    mean_revenue=("revenue", "mean")
).round(2)

print(adidas_vs_nike)

# Store the length of each description
merged_df["description_length"] = merged_df["description"].str.len()

# Upper description length limits
lengthes = [0, 100, 200, 300, 400, 500, 600, 700]

# Description length labels
labels = ["100", "200", "300", "400", "500", "600", "700"]

# Cut into bins
merged_df["description_length"] = pd.cut(merged_df["description_length"], bins=lengthes, labels=labels)

# Group by the bins
description_lengths = merged_df.groupby("description_length", as_index=False).agg(
    mean_rating=("rating", "mean"), 
    num_reviews=("reviews", "count")
).round(2)

print(description_lengths)

/tmp/ipykernel_23497/969205472.py:20: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  adidas_vs_nike = merged_df.groupby(["brand", "price_label"], as_index=False).agg(


    brand price_label  num_products  mean_revenue
0  Adidas      Budget           574       2015.68
1  Adidas     Average           655       3035.30
2  Adidas   Expensive           759       4621.56
3  Adidas       Elite           587       8302.78
4    Nike      Budget           357       1596.33
5    Nike     Average             8        675.59
6    Nike   Expensive            47        500.56
7    Nike       Elite           130       1367.45
  description_length  mean_rating  num_reviews
0                100         2.26            7
1                200         3.19          526
2                300         3.28         1785
3                400         3.29          651
4                500         3.35          118
5                600         3.12           15
6                700         3.65           15


/tmp/ipykernel_23497/969205472.py:40: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  description_lengths = merged_df.groupby("description_length", as_index=False).agg(
